In [1]:
from nltk.stem import WordNetLemmatizer 
import pandas

lemmatizer = WordNetLemmatizer()

raw = """
        DENNIS: Listen,a strange woman or man lying in ponds distributing swords is no basis for a system of government.
        Supreme executive power derives from a mandate from the masses, not from some farcical aquatic ceremony.
        """
word_list = ["DENNIS:", "strange", "woman", "mandate", "mass","love", "lie", "lying", "pond", "supreme", "is", "no", "one", "man", "you", "me", "death"]
lem_result = []
dictionary_lookup = {}

for word in raw.split(' '):
    lem_word = lemmatizer.lemmatize(word.lower())
    lem_result.append(lem_word)
    
    
for i, j in enumerate(word_list):
    if j.lower() not in lem_result:
        dictionary_lookup[i] = "ovt"
    else:
        dictionary_lookup[i] = j.lower()
        
lookup_dictionary = pandas.DataFrame(dictionary_lookup, index=[0]).transpose()
lookup_dictionary.columns = ['word']

def build_id(x):
    if x == "ovt":
        return lookup_dictionary.shape[0] + 1
    return lookup_dictionary.index[lookup_dictionary['word'] == x][0]

lookup_dictionary['ID'] = lookup_dictionary.word.apply(build_id)

In [2]:
lookup_dictionary

,word,ID
0,dennis:,0
1,strange,1
2,woman,2
3,mandate,3
4,ovt,18
5,ovt,18
6,ovt,18
7,lying,7
8,pond,8
9,supreme,9


In [3]:
from sklearn.preprocessing import OneHotEncoder

In [4]:
onehot_encoding = OneHotEncoder().fit_transform(lookup_dictionary.word.values.reshape((-1, 1))).todense()

In [5]:
onehot_encoding

matrix([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

In [6]:
from nltk.corpus import wordnet

In [7]:
for synonym in wordnet.synsets("build"):
    print("({0})".format(synonym.pos()))
    print(", ".join([l.name() for l in synonym.lemmas()]))

(n)
physique, build, body-build, habitus
(n)
human_body, physical_body, material_body, soma, build, figure, physique, anatomy, shape, bod, chassis, frame, form, flesh
(v)
construct, build, make
(v)
build_up, work_up, build, progress
(v)
build, establish
(v)
build
(v)
build
(v)
build
(v)
build
(v)
build
(v)
build_up, work_up, build, ramp_up
(v)
build


In [8]:
man = wordnet.synset("man.n.01")
man_hyper = lambda s: s.hypernyms()
list(man.closure(man_hyper))

[Synset('adult.n.01'),
 Synset('male.n.02'),
 Synset('person.n.01'),
 Synset('causal_agent.n.01'),
 Synset('organism.n.01'),
 Synset('physical_entity.n.01'),
 Synset('living_thing.n.01'),
 Synset('entity.n.01'),
 Synset('whole.n.02'),
 Synset('object.n.01')]

In [9]:
build = [
    {"sentence": "I never said she stole my money", "context_word": "I", "interpretation": "Someone else said it"},
    {"sentence": "I NEVER said she stole my money", "context_word": "NEVER", "interpretation": "I didn't say it"},
    {"sentence": "I never SAID she stole my money", "context_word": "SAID", "interpretation": "I only implied it"},
    {"sentence": "I never said SHE stole my money", "context_word": "SHE", "interpretation": "I said someone did, not necessarily her"},
    {"sentence": "I never said she STOLE my money", "context_word": "STOLE", "interpretation": "I considered it borrowed"},
    {"sentence": "I never said she stole MY money", "context_word": "MY", "interpretation": "Only that she stole money, not necessarily my own"},
    {"sentence": "I never said she stole MY MONEY", "context_word": "MONEY", "interpretation": "She stole something of mine, not my money"},
]
built = pandas.DataFrame(build)

In [10]:
sentence = built[['sentence','context_word','interpretation']]

In [11]:
sentence

,sentence,context_word,interpretation
0,I never said she stole my money,I,Someone else said it
1,I NEVER said she stole my money,NEVER,I didn't say it
2,I never SAID she stole my money,SAID,I only implied it
3,I never said SHE stole my money,SHE,"I said someone did, not necessarily her"
4,I never said she STOLE my money,STOLE,I considered it borrowed
5,I never said she stole MY money,MY,"Only that she stole money, not necessarily my own"
6,I never said she stole MY MONEY,MONEY,"She stole something of mine, not my money"
